# 보상함수 예제

<p style="display: flex; justify-content: space-between; align-items: center;">
    <b>Unit 03. 보상함수 예제</b>
    <span><b>Section 08. 보상 함수</b></span>
</p>

---
<h3 style="display: flex; justify-content: space-between; align-items: center;">
    PPT

</h3>

<div style="position:relative; width:100%; padding-bottom:65%;">
    <iframe 
        src="https://drive.google.com/file/d/1Nr_-tMlRhSoeiWSM4gnJkIwvqmLufU6G/preview" 
        style="position:absolute; top:0; left:0; width:100%; height:100%"></iframe>
</div>


---
<h3 style="display: flex; justify-content: space-between; align-items: center;">
    Video
    <span style="font-size: 0.7em;">
        <a href="../premium/01.html" target="_blank" title="이 영상은 프리미엄 과정에 등록된 계정에서만 볼 수 있습니다">🚀 프리미엄 과정 </a> 
    </span>
    <span style="font-size: 0.6em;">
        <a href="" title="인프런에서도 이 영상을 시청할 수 있습니다"><img src="https://cdn.inflearn.com/public/users/thumbnails/234401/660102d4-1e7b-4c43-a7ba-7d0ee6d96b83" height="14px" style="vertical-align: middle; padding-right: 3px;"> 인프런</a> |
        <a href="https://drive.google.com/file/d/11Y30eP8BqnHQeFxXr6WSkAwn4v2Ina3A" target="_blank">🔗 Link </a>
    </span>
</h3>

<div style="position:relative; width:100%; padding-bottom:65%;">
    <iframe 
        src="https://drive.google.com/file/d/11Y30eP8BqnHQeFxXr6WSkAwn4v2Ina3A/preview" 
        style="position:absolute; top:0; left:0; width:100%; height:100%"></iframe>
</div>


```{important}
- 영상은 **크롬 웹브라우저**를 사용할 것을 적극 권장합니다.
- 영상은 **프리미엄 과정**에 등록된 구글 계정에서만 보입니다.
  - 프리미엄 과정에 등록되었는데도 영상이 보이지 않나요?  
  - <a href="../premium/04.html" target="_blank">비디오 재생문제 해결하기</a>
- **Q&A 댓글 달기** : 영상 우측 상단 `새 탭에서 보기` 버튼을 클릭하면 영상에 질문 댓글을 작성할 수 있습니다.
```

## Resources

### Example 1
- Time trial - follow the center line
```python3
def reward_function(params):
    '''
    Example of rewarding the agent to follow center line
    '''
    
    # Read input parameters
    track_width = params['track_width']
    distance_from_center = params['distance_from_center']
    
    # Calculate 3 markers that are at varying distances away from the center line
    marker_1 = 0.1 * track_width
    marker_2 = 0.25 * track_width
    marker_3 = 0.5 * track_width
    
    # Give higher reward if the car is closer to center line and vice versa
    if distance_from_center <= marker_1:
        reward = 1.0
    elif distance_from_center <= marker_2:
        reward = 0.5
    elif distance_from_center <= marker_3:
        reward = 0.1
    else:
        reward = 1e-3  # likely crashed/ close to off track
    
    return float(reward)
```

### Example 2
- Time trial - stay inside the two borders
```python3
def reward_function(params):
    '''
    Example of rewarding the agent to stay inside the two borders of the track
    '''
    
    # Read input parameters
    all_wheels_on_track = params['all_wheels_on_track']
    distance_from_center = params['distance_from_center']
    track_width = params['track_width']
    
    # Give a very low reward by default
    reward = 1e-3

    # Give a high reward if no wheels go off the track and
    # the agent is somewhere in between the track borders
    if all_wheels_on_track and (0.5*track_width - distance_from_center) >= 0.05:
        reward = 1.0

    # Always return a float value
    return float(reward)
```

### Example 3
- Time trial - prevent zig-zag
```python3
def reward_function(params):
    '''
    Example of penalize steering, which helps mitigate zig-zag behaviors
    '''

    # Read input parameters
    distance_from_center = params['distance_from_center']
    track_width = params['track_width']
    abs_steering = abs(params['steering_angle']) # Only need the absolute steering angle

    # Calculate 3 marks that are farther and father away from the center line
    marker_1 = 0.1 * track_width
    marker_2 = 0.25 * track_width
    marker_3 = 0.5 * track_width

    # Give higher reward if the car is closer to center line and vice versa
    if distance_from_center <= marker_1:
        reward = 1.0
    elif distance_from_center <= marker_2:
        reward = 0.5
    elif distance_from_center <= marker_3:
        reward = 0.1
    else:
        reward = 1e-3  # likely crashed/ close to off track

    # Steering penality threshold, change the number based on your action space setting
    ABS_STEERING_THRESHOLD = 15 
    
    # Penalize reward if the car is steering too much
    if abs_steering > ABS_STEERING_THRESHOLD:
        reward *= 0.8
    return float(reward)
```

### Example 4
- Object avoidance and head-to-head - stay on one lane and not crashing (default for OA and h2h)
- [Link](https://docs.aws.amazon.com/deepracer/latest/developerguide/deepracer-reward-function-examples.html#deepracer-reward-function-example-3)
```python3
def reward_function(params):
    '''
    Example of rewarding the agent to stay inside two borders
    and penalizing getting too close to the objects in front
    '''

    all_wheels_on_track = params['all_wheels_on_track']
    distance_from_center = params['distance_from_center']
    track_width = params['track_width']
    objects_distance = params['objects_distance']
    _, next_object_index = params['closest_objects']
    objects_left_of_center = params['objects_left_of_center']
    is_left_of_center = params['is_left_of_center']

    # Initialize reward with a small number but not zero
    # because zero means off-track or crashed
    reward = 1e-3

    # Reward if the agent stays inside the two borders of the track
    if all_wheels_on_track and (0.5 * track_width - distance_from_center) >= 0.05:
        reward_lane = 1.0
    else:
        reward_lane = 1e-3

    # Penalize if the agent is too close to the next object
    reward_avoid = 1.0

    # Distance to the next object
    distance_closest_object = objects_distance[next_object_index]
    # Decide if the agent and the next object is on the same lane
    is_same_lane = objects_left_of_center[next_object_index] == is_left_of_center

    if is_same_lane:
        if 0.5 <= distance_closest_object < 0.8: 
            reward_avoid *= 0.5
        elif 0.3 <= distance_closest_object < 0.5:
            reward_avoid *= 0.2
        elif distance_closest_object < 0.3:
            reward_avoid = 1e-3 # Likely crashed

    # Calculate reward by putting different weights on 
    # the two aspects above
    reward += 1.0 * reward_lane + 4.0 * reward_avoid

    return reward
```